<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Разбивка-данных-на-выборки" data-toc-modified-id="Разбивка-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбивка данных на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#(бонус)-Проверка-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверка модели на адекватность</a></span></li></ul></div>

# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Построим модель с максимально большим значением *accuracy*, чтобы довести долю правильных ответов по крайней мере до 0.75.

# Описание данных

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Изучение данных

Импортируем необходимые библиотеки.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

Прочитаем и выведем первые 5 строк наших данных.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Посмотрим на общую информацию.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


Итак, в таблице 5 столбцов. Тип данных для столбцов с числовым значением - `float64` и `int64`.

Согласно документации к данным:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### Вывод

Предварительно можно сказать, что данные всех столбцов, кроме `is_ultra` распределены нормально. Столбец `is_ultra` положительно скошен. Больше всего клиенты пользуются тарифом "Смарт". 

Данные уже очищены. Можно начинать с ними работать.

## Разбивка данных на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки. 

In [6]:
# для начала разделим исходые данные на обучающую и валидационную
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=123)
# а затем валидационную выборку пополам, на валидационную и тестовую
df_valid, df_test = train_testt_split(df_valid, test_size=0.5, random_state=123)

In [7]:
df_train.shape

(1928, 5)

In [8]:
df_valid.shape

(643, 5)

In [9]:
df_test.shape

(643, 5)

Разделили исходные данные на обучающую, валидационную и тестовую в соотношении 3:1:1.

## Исследование моделей

Создадим переменные для признаков и целевого признака.

In [10]:
# обучающие переменные
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
# валидационные переменные 
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
# тестовые переменные
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

**Обучим модель дерева решений.**

In [11]:
# используем команду времени, чтобы фиксировать время работы каждой модели 
startTime_1 = time.time()

best_accuracy_1 = 0
best_model = None
# цикл проходящий по гиперпараметру - глубина дерева
for depth in range(1, 30):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_train, target_train) # обучим модель на обучающей выборке
    predictions = model.predict(features_valid) # предскажем ответы 
    accuracy = accuracy_score(target_valid, predictions) # посчитаем качество модели
    if accuracy > best_accuracy_1:
        best_accuracy_1 = accuracy # сохраним лучшее значение метрики accuracy
        best_deth = depth # сохраним глубину дерева для лучшего значения метрики accuracy
print(f'Максимальная глубина дерева: {best_deth}')
print(f'Аccuracy лучшей модели на валидационной выборке {best_accuracy_1:.3}')
    
endTime_1 = time.time()
totalTime_1 = endTime_1 - startTime_1
print(f"Время, затраченное на выполнение данного кода {totalTime_1:.3}")

Максимальная глубина дерева: 5
Аccuracy лучшей модели на валидационной выборке 0.823
Время, затраченное на выполнение данного кода 0.244


**Обучим модель случайного леса.**

In [12]:
startTime_2 = time.time()

best_accuracy_2 = 0
best_estimators = None
for est in range(1, 50):
    # обучим модель с заданным количеством деревьев
    model = RandomForestClassifier(random_state=123, n_estimators=est) 
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитаем качество модели 
    # на валидационной выборке
    if result > best_accuracy_2:
        best_estimators = est # сохраним кол-во деревьев для лучшего значения метрики accuracy
        best_accuracy_2 = result # сохраним наилучшее значение метрики accuracy
print(f'Количество деревьев {best_estimators}')
print(f'Аccuracy лучшей модели на валидационной выборке {best_accuracy_2:.3}')
    
endTime_2 = time.time()
totalTime_2 = endTime_2 - startTime_2
print(f"Время, затраченное на выполнение данного кода {totalTime_2:.3}")

Количество деревьев 38
Аccuracy лучшей модели на валидационной выборке 0.829
Время, затраченное на выполнение данного кода 4.41


**Обучим модель логистической регрессии.**

In [13]:
startTime_3 = time.time()

best_accuracy_3 = 0
model = LogisticRegression(random_state=123, solver='lbfgs')
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
if result > best_accuracy_3:
    best_accuracy_3 = result
print(f'Аccuracy лучшей модели на валидационной выборке {best_accuracy_3:.3}')
    
endTime_3 = time.time()
totalTime_3 = endTime_3 - startTime_3
print(f"Время, затраченное на выполнение данного кода {totalTime_3:.3}")

Аccuracy лучшей модели на валидационной выборке 0.759
Время, затраченное на выполнение данного кода 0.0262


### Вывод

Оценим модели по качеству (accuracy) и скорости работы.
* Качество (accuracy). 
Самое высокое качество у случайного леса: вместо одного решающего дерева используется целый оркестр.
На втором месте — решающее дерево. Самое низкое качество предсказания у логистической регрессии.
* Скорость работы. 
Высокая скорость работы у логистической регрессии: у неё меньше всего параметров.
Скорость решающего дерева тоже высокая и зависит от глубины. Случайный лес медленнее всех: чем больше деревьев, тем неторопливее работает модель.

Объединим критерии в таблицу:

In [14]:
pd.DataFrame({'Модель': ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия']
              , 'Точность': [f'{best_accuracy_1:.3}', f'{best_accuracy_2:.3}', f'{best_accuracy_3:.3}']
              , 'Скорость (сек)': [f'{totalTime_1:.3}', f'{totalTime_2:.3}', f'{totalTime_3:.3}']})

,Модель,Точность,Скорость (сек)
0,Решающее дерево,0.823,0.244
1,Случайный лес,0.829,4.41
2,Логистическая регрессия,0.759,0.0262


## Проверка модели на тестовой выборке

**Проверим модель дерева решений на тестовой выборке.**

In [15]:
startTime_4 = time.time()

model = DecisionTreeClassifier(random_state=123, max_depth=5)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
accuracy_4 = accuracy_score(target_test, predictions)
    
print(f'Аccuracy на тестовой выборке {accuracy_4:.3}')
    
endTime_4 = time.time()
totalTime_4 = endTime_4 - startTime_4
print(f"Время, затраченное на выполнение данного кода {totalTime_4:.3}")

Аccuracy на тестовой выборке 0.771
Время, затраченное на выполнение данного кода 0.00701


**Проверим модель случайного леса на тестовой выборке.**

In [16]:
startTime_5 = time.time()

model = RandomForestClassifier(random_state=123, n_estimators=22) 
model.fit(features_train, target_train)
predictions = model.predict(features_test)
accuracy_5 = accuracy_score(target_test, predictions)

print(f'Аccuracy на тестовой выборке {accuracy_5:.3}')
    
endTime_5 = time.time()
totalTime_5 = endTime_5 - startTime_5
print(f"Время, затраченное на выполнение данного кода {totalTime_5:.3}")

Аccuracy на тестовой выборке 0.796
Время, затраченное на выполнение данного кода 0.0974


**Проверим модель логистической регрессии на тестовой выборке.**

In [17]:
startTime_6 = time.time()

model = LogisticRegression(random_state=12345, solver='lbfgs')
model.fit(features_train, target_train) 
predictions = model.predict(features_test)
accuracy_6 = accuracy_score(target_test, predictions)

print(f'Аccuracy на тестовой выборке {accuracy_6:.3}')
    
endTime_6 = time.time()
totalTime_6 = endTime_6 - startTime_6
print(f"Время, затраченное на выполнение данного кода {totalTime_6:.3}")

Аccuracy на тестовой выборке 0.731
Время, затраченное на выполнение данного кода 0.0249


### Вывод

Точность на тестовой выборке всех моделей стала немного меньше. Мы столкнулись с "легким" переобучением. Время работы моделей сильно снизилось, кроме логистической регрессии, у которой осталось почти неизменной. 

Значение accuracy удалось достичь не меньше 0.75.

Объединим критерии при валидационной и тестовой выборке в таблицу:

In [18]:
pd.DataFrame({'Модель': ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия']
              , 'Точность': [f'{best_accuracy_1:.3}', f'{best_accuracy_2:.3}', f'{best_accuracy_3:.3}']
              , 'Скорость (сек)': [f'{totalTime_1:.3}', f'{totalTime_2:.3}', f'{totalTime_3:.3}']
              , 'Точность_тест': [f'{accuracy_4:.3}', f'{accuracy_5:.3}', f'{accuracy_6:.3}']
              , 'Скорость (сек)_тест': [f'{totalTime_4:.3}', f'{totalTime_5:.3}', f'{totalTime_6:.3}']})

,Модель,Точность,Скорость (сек),Точность_тест,Скорость (сек)_тест
0,Решающее дерево,0.823,0.244,0.771,0.00701
1,Случайный лес,0.829,4.41,0.796,0.0974
2,Логистическая регрессия,0.759,0.0262,0.731,0.0249


## (бонус) Проверка модели на адекватность

In [38]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)
accuracy = accuracy_score(target_test, dummy_pred)

print(f'Аccuracy равно {accuracy:.4}')


Аccuracy равно 0.6936
